#GAN with semantics

In [ ]:
! mkdir /content/cityScapes

In [ ]:
! wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=d.a.vandendoel@students.uu.nl&password=Derkojo95!&submit=Login' https://www.cityscapes-dataset.com/login/
! wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1 -P /content/cityScapes
! wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3 -P /content/cityScapes

In [ ]:
! unzip /content/cityScapes/gtFine_trainvaltest.zip -d /content/cityScapes/annotations
! unzip /content/cityScapes/leftImg8bit_trainvaltest.zip -d /content/cityScapes/img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow-addons
!pip install -U tensorboard_plugin_profile

In [ ]:
#from __future__ import print_function, division
import cv2
import datetime
import itertools
import os
import PIL
import scipy
import sys
import time
import json

import numpy as np 
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from skimage import transform
from tqdm import tqdm
from random import randint, seed
from google.colab.patches import cv2_imshow
from imageio import imread
from tensorflow.keras import layers
from glob import glob
from IPython import display
from collections import namedtuple
import seaborn as sns
from matplotlib.colors import LogNorm, Normalize
from sklearn.metrics import confusion_matrix

#New mask function (Do not use)

In [ ]:
def mask_patch(image, patchsize, location_h, location_w):
  image_h, image_w, image_d = image.shape #batch, height, width, depth
  patch_h, patch_w = patchsize
  patch = tf.ones((patch_h, patch_w, 3))

  pad_left = tf.zeros((patch_h, location_w, 3)) #pad the image with zeros from the left border of the image until the patch
  pad_right = tf.zeros((patch_h, image_w-(location_w+patch_w), 3)) #pad the image with zeros from the patch until the right border of the image
  
  print(f"pad left shape: {pad_left.shape}, patch shape: {patch.shape}, pad right shape: {pad_right.shape}")

  new_patch = tf.concat([pad_left, patch, pad_right], axis =1)
  new_patch_w = new_patch.shape[1]

  pad_up = tf.zeros((location_h, new_patch_w, 3))
  pad_down = tf.zeros((image_h - (location_h+patch_h), new_patch_w, 3))
  
  new_patch = tf.concat([pad_up, new_patch, pad_down], axis=0)

  return new_patch

In [ ]:
def image_mask(image, mask_height, mask_width, num_masks, rand_seed = None):
  # random_cutout function needs mask_height * mask_width to be divisible by 2
  if (mask_height * mask_width)%2 != 0:
    raise Exception("Error! mask size must be divisible by 2")
  

  im_height = tf.cast(IM_HEIGHT, dtype=tf.dtypes.int32)
  im_width = tf.cast(IM_WIDTH, dtype=tf.dtypes.int32)  
  mask_height = tf.cast(mask_height, dtype=tf.dtypes.int32)
  mask_width = tf.cast(mask_width, dtype=tf.dtypes.int32)
  
  min_height = tf.cast(tf.math.round(im_height/5), tf.dtypes.int32)
  min_width = tf.cast(tf.math.round(mask_width/2), tf.dtypes.int32)

  try:
    image_shape = image.shape
  except:
    image_shape = image.get_shape()

  if len(image_shape) == 3:
    image = tf.expand_dims(image, 0)
  elif len(image_shape) == 4:
    pass
  else:
    raise Exception('Error! Tensor shape must either be: (batch_size, image height, image width, color channels) or (image height, image width, color channels)')
  
  mask = tf.ones_like(image, tf.dtypes.float32)[0,:,:,0:1]

  for i in range(num_masks):
    height_offset = tf.random.uniform(shape = (), minval= min_height, maxval=im_height-mask_height, dtype=tf.dtypes.int32, seed=rand_seed)
    width_offset = tf.random.uniform(shape = (), minval= 0, maxval= im_width-mask_width, dtype=tf.dtypes.int32, seed=rand_seed)
    print(f"height_offset is: {height_offset}, width_offset: {width_offset}, mask_height: {mask_height}, mask_width: {mask_width}")
    mask = mask_patch(mask, (mask_height, mask_width),  height_offset, width_offset)
  
  masked_image = (1-mask) * image

  return masked_image[0], mask[0]

#Load JSON

In [ ]:
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # 'preparation' folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,        0 , 'flat'            , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,        1 , 'flat'            , 1       , False        , False        , (246, 35,232) ), #244,35,232
    Label(  'parking'              ,  9 ,      255 , 'flat'            , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      255 , 'flat'            , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,        2 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,        3 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,        4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      255 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      255 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,        5 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      255 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,        6 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,        7 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,        8 , 'nature'          , 4       , False        , False        , (107,142, 35) ), #107 142 35
    Label(  'terrain'              , 22 ,        9 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,       10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,       11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,       12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,       13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,       14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,       15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,       16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,       17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,       18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'license plate'        , -1 ,       -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]

#convert rgb to grayscale values, this produces a unique grayscale value for each label
def rgb_gray(rgb):
  r, g ,b = rgb
  return 0.2989*r + 0.5870*g + 0.1140*b

#create dictionary with grayscale as key and id as value
color_id_list = []
color_id = {}
id_r = {}
id_g = {}
id_b = {}
for label in labels:
  key_list = tf.image.rgb_to_grayscale(tf.convert_to_tensor(label.color))
  key = int(rgb_gray(label.color))
  color_id_list.append((key_list, label.id))
  color_id[key] = label.id
  id_r[label.id] = label.color[0]
  id_g[label.id] = label.color[1]
  id_b[label.id] = label.color[2]
color_id[0] = 0
color_id[16] = 26 # grayscale value 16 has category "car"


#tf.image.rgb_to_grayscale
#convert dictionary to tf StaticHashTable
init = tf.lookup.KeyValueTensorInitializer(tf.convert_to_tensor(list(color_id.keys())), tf.convert_to_tensor((list(color_id.values()))))
gray2label = tf.lookup.StaticHashTable(init,default_value= -100)

init = tf.lookup.KeyValueTensorInitializer(tf.convert_to_tensor(list(id_r.keys())), tf.convert_to_tensor((list(id_r.values()))))
label2r =  tf.lookup.StaticHashTable(init,default_value= -100)

init = tf.lookup.KeyValueTensorInitializer(tf.convert_to_tensor(list(id_g.keys())), tf.convert_to_tensor((list(id_g.values()))))
label2g =  tf.lookup.StaticHashTable(init,default_value= -100)

init = tf.lookup.KeyValueTensorInitializer(tf.convert_to_tensor(list(id_b.keys())), tf.convert_to_tensor((list(id_b.values()))))
label2b =  tf.lookup.StaticHashTable(init,default_value= -100)

# Image pre-processing functions

In [ ]:
IM_HEIGHT = 256
IM_WIDTH = 512
BUFFER_SIZE = 2975
BATCH_SIZE = 1
OUTPUT_CHANNELS = 3
LAMBDA = 100
MASK_HEIGTH = 64
MASK_WIDTH = 32

In [ ]:
def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.io.decode_png(image, channels=3)
  image = tf.cast(image, tf.float32)

  return image

In [ ]:
def resize(input_image, height, width):
    image = tf.image.resize(
        input_image, 
        [height, width],
        method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
    )
  
    return image

In [ ]:
def normalize(image):
  image = image / 127.5 - 1
  return image

In [ ]:
def image_mask(image, mask_height, mask_width, num_masks, rand_seed = None):
  # random_cutout function needs mask_height * mask_width to be divisible by 2
  if (mask_height * mask_width)%2 != 0:
    raise Exception("Error! mask size must be divisible by 2")
  
  im_height = tf.cast(IM_HEIGHT, dtype=tf.dtypes.int32)
  im_width = tf.cast(IM_WIDTH, dtype=tf.dtypes.int32)  
  mask_height = tf.cast(mask_height, dtype=tf.dtypes.int32)
  mask_width = tf.cast(mask_width, dtype=tf.dtypes.int32)
  
  min_height = tf.cast(tf.math.round((mask_height/2)+15), tf.dtypes.int32)
  min_width = tf.cast(tf.math.round((mask_width/2)+10), tf.dtypes.int32)
  max_height = tf.cast(tf.math.round(im_height/10), tf.dtypes.int32)

  try:
    image_shape = image.shape
  except:
    image_shape = image.get_shape()

  if len(image_shape) == 3:
    image = tf.expand_dims(image, 0)
  elif len(image_shape) == 4:
    pass
  else:
    raise Exception('Error! Tensor shape must either be: (batch_size, image height, image width, color channels) or (image height, image width, color channels)')
  
  mask = tf.zeros_like(image, tf.dtypes.float32)[:,:,:,0:1]
  locations = []

  #create masks
  for i in range(num_masks):
    mask_height_random = tf.random.uniform(shape = (), minval= mask_height-5, maxval=mask_height+15, dtype=tf.dtypes.int32, seed=rand_seed)
    mask_height_random = mask_height_random if mask_height_random%2 == 0 else mask_height_random-1
    mask_width_random = tf.random.uniform(shape = (), minval= mask_width, maxval=mask_width+10, dtype=tf.dtypes.int32, seed=rand_seed)
    mask_width_random = mask_width_random if mask_width_random%2 == 0 else mask_width_random-1
    
    height_offset = tf.random.uniform(shape = (), minval= min_height, maxval=im_height-max_height, dtype=tf.dtypes.int32, seed=rand_seed)
    width_offset = tf.random.uniform(shape = (), minval= min_width, maxval= im_width-min_width, dtype=tf.dtypes.int32, seed=rand_seed)
    mask = tfa.image.cutout(mask, (mask_height_random, mask_width_random), (height_offset, width_offset), constant_values = 1)
    locations.append((mask_height_random, mask_width_random, height_offset, width_offset))
  
  masked_image = (1-mask) * image

    #mask = tf.zeros([image_heigt, image_width])
    #mask = mask[height_offset:height_offset+mask_height].assign(tf.ones([mask_height, mask_width]))
    #image = mask * image


  return masked_image[0], mask[0], locations

In [ ]:
def label_to_rgb(labeled_im):
  r = label2r.lookup(labeled_im)[:,:,:,0]
  g = label2g.lookup(labeled_im)[:,:,:,0]
  b = label2b.lookup(labeled_im)[:,:,:,0]

  image = tf.stack([r,g,b], axis = 3)
  image = tf.cast(image, tf.float32)
  image = normalize(image)

  return image

In [ ]:
def load_images(semantics_file, label_file):
  semantics = load(semantics_file)
  semantics = resize(semantics, 256, 512)
  semantics = normalize(semantics)
  masked_semantics, mask, locations = image_mask(semantics, MASK_HEIGTH, MASK_WIDTH, 4)
  
  mask = tf.cast(mask, tf.int32)
  label = tf.io.read_file(label_file)
  label = tf.io.decode_png(label, channels = 0)
  label = tf.cast(label, tf.int32)
  label = resize(label, 256, 512)
  masked_label = (1-mask)*label

  return masked_label, label, semantics, masked_semantics, mask, label_file, locations

#Prepare data

In [ ]:
# Fill pandas dataframe with the paths to the files
def folder_to_pd(semantics_dir):
  semantics_paths = {"test": [], "train": [], "val": []}
  label_paths = {"test": [], "train": [], "val": []}

  for data_type in ["test", "train", "val"]:
    semantics_subdir = os.path.join(semantics_dir, data_type)
    for root, subdir, files in os.walk(semantics_subdir):
      subdir.sort()
      files.sort()
      if files:
        for i,file in enumerate(files):
          if file[-9:] == "color.png":
            semantics_paths[data_type].append(os.path.join(root, file))
          if file[-12:] == "labelIds.png":
            label_paths[data_type].append(os.path.join(root, file))
  
  
  test_ds = pd.DataFrame(list(zip(semantics_paths["test"], label_paths["test"])), columns =['Semantics', 'label'])
  train_ds = pd.DataFrame(list(zip(semantics_paths["train"], label_paths["train"])), columns =['Semantics', 'label'])
  val_ds = pd.DataFrame(list(zip(semantics_paths["val"], label_paths["val"])), columns =['Semantics', 'label'])

  return test_ds, train_ds, val_ds

test_ds, train_ds, val_ds = folder_to_pd("/content/cityScapes/annotations/gtFine")

test_ds = tf.data.Dataset.from_tensor_slices((test_ds["Semantics"], test_ds["label"]))
train_ds = tf.data.Dataset.from_tensor_slices((train_ds["Semantics"], train_ds["label"]))
val_ds = tf.data.Dataset.from_tensor_slices((val_ds["Semantics"], val_ds["label"]))

In [ ]:
tf_train = train_ds.shuffle(train_ds.cardinality(), reshuffle_each_iteration=False)
#tf_train = tf_train.interleave(tf.data.TFRecordDataset, num_parallel_calls = tf.data.AUTOTUNE)
tf_train = tf_train.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
tf_train = tf_train.batch(BATCH_SIZE)
tf_train = tf_train.cache("/content/temporary.tfcache")
tf_train = tf_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
tf_val = val_ds.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
tf_val = tf_val.batch(BATCH_SIZE)
tf_val = tf_val.prefetch(tf.data.AUTOTUNE)

In [ ]:
for masked_label, label, image, masked_image, mask, image_file, locations in tf_val.take(1):
  plt.imshow(label_to_rgb(label)[0]*0.5+0.5)

In [ ]:
plt.imshow(label_to_rgb(masked_label)[0]*0.5+0.5)

#Label GAN

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
        tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                                kernel_initializer=initializer, use_bias=False))

    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())
        result.add(tf.keras.layers.Lambda(lambda x: tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)))

    result.add(tf.keras.layers.LeakyReLU())

    return result

In [ ]:
def upsample(filters, size, h, w, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()

    result.add(
        tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                        padding='same',
                                        kernel_initializer=initializer,
                                        use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())
    result.add(tf.keras.layers.Lambda(lambda x: tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)))

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result

In [ ]:
def GeneratorSemantics():
        down_stack = [
            downsample(64, 4, apply_batchnorm=False),  # (bs, 128, 256, 64)
            downsample(128, 4),  # (bs, 64, 128, 128)
            downsample(256, 4),  # (bs, 32, 64, 256)
            downsample(512, 4),  # (bs, 16, 32, 512)
            downsample(512, 4),  # (bs, 8, 16, 512)
            downsample(512, 4),  # (bs, 4, 8, 512)
            downsample(512, 4),  # (bs, 2, 4, 512)
            downsample(512, 4),  # (bs, 1, 2, 512)
        ]

        up_stack = [
            upsample(512, 4, 1, 2, apply_dropout=True),  # (bs, 2, 4, 1024)
            upsample(512, 4, 2, 4, apply_dropout=True),  # (bs, 4, 8, 1024)
            upsample(512, 4, 4, 8, apply_dropout=True),  # (bs, 8, 16, 1024)
            upsample(512, 4, 8, 16),  # (bs, 16, 32, 1024)
            upsample(256, 4, 16, 32),  # (bs, 32, 64, 512)
            upsample(128, 4, 32, 64),  # (bs, 64, 128, 256)
            upsample(64, 4, 64, 128),  # (bs, 128, 256, 128)
        ]

        initializer = tf.random_normal_initializer(0., 0.02)
        last_upsample = upsample(64, 4, 128, 256)
        last = tf.keras.layers.Conv2D(35, 4, strides=1, padding='same', kernel_initializer=initializer, activation='tanh')  # (bs, 256, 512, 35) 35 is the number of classes 

        concat = tf.keras.layers.Concatenate()
        semantics = tf.keras.layers.Input(shape=[IM_HEIGHT, IM_WIDTH, 1])

        x = semantics

        # Downsampling through the model
        skips = []
        for down in down_stack:
            x = down(x)
            skips.append(x)

        skips = reversed(skips[:-1])

        # Upsampling and establishing the skip connections
        for up, skip in zip(up_stack, skips):
            x = up(x)
            x = concat([x, skip])

        x = last_upsample(x)
        x = last(x)

        return tf.keras.Model(inputs=semantics, outputs=x)

In [ ]:
def DiscriminatorSemantics():
    initializer = tf.random_normal_initializer(0., 0.02)

    semantics = tf.keras.layers.Input(shape=[IM_HEIGHT, IM_WIDTH, 1], name='semantics')
    tar = tf.keras.layers.Input(shape=[IM_HEIGHT, IM_WIDTH, 1], name='target_image')

    x = tf.keras.layers.concatenate([semantics, tar])  # (bs, 256, 512, channels*2)

    down1 = downsample(64, 4, False)(x)  # (bs, 128, 256, 64)
    down2 = downsample(128, 4)(down1)  # (bs, 64, 128, 128)
    down3 = downsample(256, 4)(down2)  # (bs, 32, 64, 256)

    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (bs, 34, 66, 256)
    conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                  kernel_initializer=initializer,
                                  use_bias=False)(zero_pad1)  # (bs, 31, 63, 512)

    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (bs, 33, 65, 512)

    last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                  kernel_initializer=initializer)(zero_pad2)  # (bs, 30, 62, 1)

    return tf.keras.Model(inputs=[semantics, tar], outputs=last)

In [ ]:
generator_semantics = GeneratorSemantics()
discriminator_semantics = DiscriminatorSemantics()

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
  # Sparse Categorical Crossentropy
  CE_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(target, gen_output)
  total_gen_loss = gan_loss + CE_loss

  return total_gen_loss, gan_loss, CE_loss

def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [ ]:
lr = 2e-4
generator_optimizer_semantics = tf.keras.optimizers.legacy.Adam(lr, beta_1=0.75)
discriminator_optimizer_semantics = tf.keras.optimizers.legacy.Adam(lr, beta_1=0.75)

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/Scriptie/ckpt/SemanticInpainting'
checkpoint_prefix = os.path.join(checkpoint_dir, "semantics", "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer_semantics,
                                 discriminator_optimizer=discriminator_optimizer_semantics,
                                 generator=generator_semantics,
                                 discriminator=discriminator_semantics)

In [ ]:
checkpoint.restore("/content/drive/MyDrive/Scriptie/ckpt/SemanticInpainting/semantics/ckpt-1")

In [ ]:
def gen_to_label(gen_output):
  return tf.expand_dims(tf.math.argmax(gen_output, axis = -1, output_type = tf.dtypes.int32), -1)

In [ ]:
def label_to_sparse(labels):
  return tf.squeeze(tf.one_hot(tf_max, depth = 35),3)

# Generate images and train_step functions

In [ ]:
def generate_images(model, input, tar, mask):  
  prediction = model(input, training=True)
  plt.figure(figsize=(15, 15))

  prediction = tf.math.argmax(prediction, axis = -1, output_type = tf.dtypes.int32)
  prediction = label_to_rgb(tf.expand_dims(prediction, -1))
  
  input = tf.cast(input, tf.int32)
  input = label_to_rgb(input)


  display_list = [input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

#Fit (epochs)

In [ ]:
@tf.function
def train_epoch(input_image, mask, target):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator_semantics(input_image, training=True)
    gen_output_argmax = gen_to_label(gen_output) #convert the 35 channel output of the generator to a 1 channel output with labels

    disc_real_output = discriminator_semantics([input_image, target], training=True)
    disc_generated_output = discriminator_semantics([input_image, gen_output_argmax], training=True) #discriminator takes predicted labels

    gen_total_loss, gen_gan_loss, gen_CE_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  # from inpainted patches, count pixels that are not equal to the ground truth
  mask_size = tf.math.count_nonzero(mask)
  gen_acc = tf.math.count_nonzero(tf.math.not_equal(target*mask, tf.expand_dims(gen_output_argmax, -1)*mask))/mask_size

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator_semantics.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                              discriminator_semantics.trainable_variables)

  generator_optimizer_semantics.apply_gradients(zip(generator_gradients, generator_semantics.trainable_variables))
  discriminator_optimizer_semantics.apply_gradients(zip(discriminator_gradients, discriminator_semantics.trainable_variables))
  
  return gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc
    
  
def getValAccLoss(input_image, mask, target): 
  gen_output = generator_semantics(input_image, training=True)
  gen_output_argmax = gen_to_label(gen_output) #convert the 35 channel output of the generator to a 1 channel output with labels

  disc_real_output = discriminator_semantics([input_image, target], training=True)
  disc_generated_output = discriminator_semantics([input_image, gen_output_argmax], training=True) #discriminator takes predicted labels

  gen_total_loss, gen_gan_loss, gen_CE_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
  disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  mask_size = tf.math.count_nonzero(mask)
  gen_acc = tf.math.count_nonzero(tf.math.not_equal(target*mask, tf.expand_dims(gen_output_argmax, -1)*mask))/mask_size

  return gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc

In [ ]:
log_dir= '/content/drive/MyDrive/Scriptie/logs/SemanticInpainting/epoch/'
summary_writer_train = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_Train")
summary_writer_val = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_Val")

In [ ]:
def fit(train_ds, test_ds, epochs):
  masked_label_test, label_test, image_target, image_masked_test, mask_test, path = next(iter(test_ds.take(1)))
  start = time.time()
  total_ims = tf.cast(train_ds.cardinality(), tf.float32)
  total_ims_val = tf.cast(test_ds.cardinality(), tf.float32)

  for epoch in range(epochs):
    metrics_dict = {"gen_total_loss":0, "gen_gan_loss":0, "gen_CE_loss":0, "disc_loss":0, "gen_acc":0}
    generate_images(generator_semantics, masked_label_test, image_target, mask_test)

    print("Start training of epoch: ", epoch+1)
    for step,(masked_label, label, image, masked_image, mask, image_file) in train_ds.enumerate():
      gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc = train_epoch(masked_label, mask, label)
      metrics_list = (gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc)
      
      for i,(key,value) in enumerate(metrics_dict.items()):
        metrics_dict[key] = value + metrics_list[i]
      
      if (step+1) % 50 == 0:
        print('.', end='', flush=True)

    with summary_writer_train.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict["gen_total_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict["gen_gan_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_CE_loss', metrics_dict["gen_CE_loss"]/total_ims, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict["disc_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_acc', metrics_dict["gen_acc"]/tf.cast(total_ims, tf.float64), step=epoch)


    # Gather the metrics for the validation set
    metrics_dict_val = {"gen_total_loss":0, "gen_gan_loss":0, "gen_CE_loss":0, "disc_loss":0, "gen_acc":0}
    print("Gathering validation set metrics...")
    for step,(masked_label, label, image, masked_image, mask, image_file) in test_ds.enumerate():
      gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc = getValAccLoss(masked_label, mask, label)
      metrics_list_val = (gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss, gen_acc)
        
      for i,(key,value) in enumerate(metrics_dict_val.items()):
        metrics_dict_val[key] = value + metrics_list_val[i]
    
    with summary_writer_val.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict_val["gen_total_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict_val["gen_gan_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_CE_loss', metrics_dict_val["gen_CE_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict_val["disc_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_acc', metrics_dict_val["gen_acc"]/tf.cast(total_ims_val, tf.float64), step=epoch)

    display.clear_output(wait=True)
    print(f'Time taken for 1 epoch: {time.time()-start:.2f} sec\n')
    start = time.time()

    # Save (checkpoint) the model every 5 epochs
    #if epoch % 10 == 0 & epoch !=0: 
    #  checkpoint.save(file_prefix=checkpoint_prefix)

# Train

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir {log_dir}

In [ ]:
fit(tf_train, tf_val, 10)

In [ ]:
prediction = gen_to_label(generator_semantics(masked_label))

In [ ]:
plt.imshow(label_to_rgb(masked_label)[0]*0.5+0.5)

In [ ]:
plt.imshow(label_to_rgb(tf.expand_dims(prediction, -1))[0]*0.5+0.5)

In [ ]:
plt.imshow(label_to_rgb(label)[0]*0.5+0.5)

#Confusion matrix

In [ ]:
for masked_label, label, image, masked_image, mask, image_file, locations in tf_train.take(1):
  prediction = gen_to_label(generator_semantics(masked_label, training = True)) * mask + label * (1-mask)
  fig, axs = plt.subplots(1, 2, figsize=(15, 15))
  axs[0].imshow(label_to_rgb(label)[0]*0.5+0.5)
  axs[1].imshow(label_to_rgb(prediction)[0]*0.5+0.5)
  plt.axis('off')
  plt.show()

In [ ]:
plt.imshow(label_to_rgb(masked_label)[0]*0.5+0.5)

In [ ]:
valid_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
IoU = tf.keras.metrics.IoU(num_classes=len(valid_labels), target_class_ids=valid_labels)

In [ ]:
un, _ = tf.unique(inp)

In [ ]:
prediction_mask = tf.boolean_mask(prediction, mask)
label_mask = tf.boolean_mask(label, mask)

In [ ]:
label_flat = tf.reshape(label, [-1])
prediction_flat = tf.reshape(prediction, [-1])

In [ ]:
cfm = tf.math.confusion_matrix(label_mask, prediction_mask, num_classes=34)

In [ ]:
for step, (masked_label, label, image, masked_image, mask, image_file, locations) in tf_val.enumerate():
  prediction = gen_to_label(generator_semantics(masked_label, training = True)) * mask + label * (1-mask)
  prediction_mask = tf.boolean_mask(prediction, mask)
  label_mask = tf.boolean_mask(label, mask)
  
  if step == 0:
    pred = prediction_mask
    inp = label_mask
  else:
    pred = tf.concat((pred, prediction_mask), axis=0)
    inp = tf.concat((inp, label_mask), axis=0)

  if step%50 == 0:
    print('.', end='', flush=True)

In [ ]:
# filter the vectors such that labels that should be removed are not taken into consideration
# Find the indices of the elements to remove
def filter_vectors(tf1, tf2, index):
  mask = tf.logical_not(tf.logical_or(tf.equal(tf1, index), tf.equal(tf2, index)))
  indices_to_keep = tf.where(mask)[:, 0]

  # Remove the elements from both tensors
  tf1_filtered = tf.gather(tf1, indices_to_keep)
  tf2_filtered = tf.gather(tf2, indices_to_keep)
  return tf1_filtered, tf2_filtered

In [ ]:
inp_filtered = inp
pred_filtered = pred
remove_labels = [0, 1, 2, 3, 4, 5]

for i in remove_labels:
  inp_filtered, pred_filtered = filter_vectors(inp_filtered, pred_filtered, i)

In [ ]:
un, _ = tf.unique(inp)
un2, _ = tf.unique(pred)
print(tf.sort(un).numpy())
print(tf.sort(un2).numpy())

In [ ]:
confusion_mat = confusion_matrix(inp.numpy(), pred.numpy())
total_labels = inp_filtered.shape
confusion_mat_norm = confusion_mat/total_labels
confusion_mat_norm[confusion_mat_norm == 0.0] = np.nan

In [ ]:
cfm_filtered = tf.math.confusion_matrix(inp_filtered, pred_filtered, num_classes=34)
total_labels = inp_filtered.shape
cfm_normalized = cfm_filtered.numpy()/total_labels
cfm_normalized[cfm_normalized == 0.0] = np.nan

#threshold = 100
#cfm_thresholded = tf.where(cfm_filtered < threshold, 0, cfm_filtered)

In [ ]:
vmin = 0
vmax = np.nanmax(confusion_mat_norm)

plt.figure(figsize = (10,10))
heatmap = sns.heatmap(confusion_mat_norm, annot=False, cmap="crest", linewidths = 0.2, linecolor = "black", vmin=0, vmax=vmax)
heatmap.set(title = "Validation dataset", xlabel='Predicted label', ylabel='Ground truth label')
pos_x, textvals = plt.xticks()
pos_y, textvals = plt.yticks()

plt.xticks(ticks=pos_x, labels=keep_indices.tolist())
plt.yticks(ticks=pos_y, labels=keep_indices.tolist())
plt.show()

In [ ]:
keep_indices = tf.sort(un2).numpy()

In [ ]:
inp_filtered.numpy().shape[0]

In [ ]:
plt.figure(figsize = (10,10))
num_labels = inp_filtered.numpy().shape[0]
vmin = 0
vmax = np.max(confusion_mat/num_labels)
heatmap = sns.heatmap(confusion_mat_norm, annot=False, cmap="crest", linewidths = 0.2, linecolor = "black")
heatmap.set(title = "Validation dataset", xlabel='Predicted label', ylabel='Ground truth label')
pos_x, textvals = plt.xticks()
pos_y, textvals = plt.yticks()

plt.xticks(ticks=pos_x, labels=valid_labels)
plt.yticks(ticks=pos_y, labels=valid_labels)
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
heatmap = sns.heatmap(cfm_filtered, annot=False, cmap="crest", linewidths = 0.2, linecolor = "black", norm=LogNorm())
heatmap.set(title = "Validation dataset", xlabel='Predicted label', ylabel='Ground truth label')
pos_x, textvals = plt.xticks()
pos_y, textvals = plt.yticks()

plt.xticks(ticks=pos_x, labels=tf.sort(un2).numpy().numpy().tolist())
plt.yticks(ticks=pos_y, labels=keep_indices.numpy().tolist())
plt.show()

In [ ]:
#[row, column]
cfm_reduced.numpy()[:,15]

In [ ]:
print(cfm.numpy()[0,:])

In [ ]:
print(cfm.numpy()[24,24])

In [ ]:
heatmap.figure.savefig("sns-heatmap.png")

In [ ]:
%matplotlib inline

In [ ]:
plt.savefig('/content/heatmap.png')

In [ ]:
for masked_image, image, semantics_masked, semantics, mask, image_file, locations in tf_val.take(1)
    semantics_sparse_test = label_to_sparse(semantics)
    predicted_RGB = generatorSEMGAN([masked_image, semantics_sparse_test], training = True)
    predicted_RGB = mask * predicted_RGB + (1-mask) * image

    plt.imshow(predicted_RGB)

# Metrics

In [ ]:
def metrics(image_1, image_2):
  # calculate L1 distance
  L1_distance = tf.math.reduce_sum(tf.math.abs(image_1 - image_2))
  ssim = tf.image.ssim(image_1, image_2, 255)
  psnr = tf.image.psnr(image_1, image_2, 255)
  return [L1_distance.numpy(), ssim.numpy()[0], psnr.numpy()[0]]

In [ ]:
def splitMaskMetrics(target, prediction, locations):
  locations = np.squeeze(locations.numpy(), axis = 0)
  ssim_list = []
  psnr_list = []
  l1_list = []

  for location in locations:
    mask_height = tf.cast(location[0]/2, tf.int32).numpy()
    mask_width = tf.cast(location[1]/2, tf.int32).numpy()
    height_offset = location[2]
    width_offset = location[3]

    targetMask = target[:,height_offset-mask_height:height_offset+mask_height, width_offset-mask_width:width_offset+mask_width,:]
    predictiontMask = prediction[:, height_offset-mask_height:height_offset+mask_height, width_offset-mask_width:width_offset+mask_width, :] 
    
    ssim = tf.image.ssim(targetMask, predictiontMask, 255)
    psnr = tf.image.psnr(targetMask, predictiontMask, 255)
    l1 = tf.math.reduce_sum(tf.math.abs(targetMask-predictiontMask))


    ssim_list.append(ssim.numpy()[0])
    psnr_list.append(psnr.numpy()[0])
    l1_list.append(l1.numpy())

  return [sum(l1_list)/len(l1_list), sum(ssim_list)/len(ssim_list), sum(psnr_list)/len(psnr_list)]

In [ ]:
def IoU(label, masked_label):
  _, shape_h, shape_w, _ = label.shape
  unique_labels, _ = tf.unique(tf.reshape(label, shape_h*shape_w))
  IoU = tf.keras.metrics.IoU(num_classes=35, target_class_ids = unique_labels)(label, masked_label)
  return IoU.numpy()

In [ ]:
def get_dataset_metrics(ds):
  #input_image is the inpainted image, GT is the groundtruth
  names = ["L1","SSIM","PSNR"]
  metrics_list = []
  IoU_list = []
  img_path_list = []
  mask_list = []
  img_list = []
  semantics_list = []
  
  for (masked_image, image, semantics_masked, semantics, mask, image_file, locations) in ds:
    semantics_predicted = gen_to_label(generator_semantics(semantics_masked, training=True))
    semantics_predicted = semantics_predicted*tf.cast(mask, tf.int32)+semantics*(1-tf.cast(mask, tf.int32))
    semantics_sparse_test = label_to_sparse(semantics_predicted)

    predicted_RGB = generatorSEMGAN([masked_image, semantics_sparse_test], training = True)
    predicted_RGB = mask * predicted_RGB + (1-mask) * image

    metrics_list.append(splitMaskMetrics(image, predicted_RGB, locations))
    IoU_list.append(IoU(semantics, semantics_predicted))
    img_path_list.append(image_file.numpy()[0].decode("utf-8"))
    img_list.append(predicted_RGB)
    mask_list.append(mask)
    semantics_list.append(semantics_predicted)
  
  metrics_df = pd.DataFrame(metrics_list)
  metrics_df.columns = names
  metrics_df["IoU"] = IoU_list
  metrics_df["Img Path"] = img_path_list
  metrics_df["Image"] = img_list
  metrics_df["Mask"] = mask_list
  metrics_df["Semantics"] = semantics_list
  return metrics_df

In [ ]:
def show_image(pd_dataframe, index):
  prediction = pd_dataframe["Image"][index]
  mask = pd_dataframe["Mask"][index]
  semantics = label_to_rgb(pd_dataframe["Semantics"][index])
  gt = normalize(resize(load(pd_dataframe["Img Path"][index]), 256, 512))
  gt = tf.expand_dims(gt, 0)
  masked = (1-mask)*gt
  

  images = [masked, gt, prediction, semantics]
  title = ["Input Image", "Ground Truth", "Prediction", "Semantics"]

  plt.figure(figsize=(25, 25))
  for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(images[i][0] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()
  print("L1: ", pd_dataframe["L1"][index], 
        "SSIM: ",pd_dataframe["SSIM"][index], 
        "PSNR: ",pd_dataframe["PSNR"][index])

In [ ]:
metrics_test_SEMGAN = get_dataset_metrics(tf_val)

In [ ]:
show_image(metrics_test_SEMGAN, 21)

In [ ]:
metrics_test_SEMGAN[["L1","SSIM","PSNR", "IoU"]]

In [ ]:
metrics_test_SEMGAN.mean()

In [ ]:
metrics_test_SEMGAN.mean()

In [ ]:
def reconstruct_img(label_file, mask):
  label = tf.io.read_file(label_file)
  label = tf.io.decode_png(label, channels = 0)
  label = tf.cast(label, tf.int32)
  label = resize(label, 256, 512)
  masked_label = (1-mask)*label
  

  return label, masked_label, mask

In [ ]:
def show_img(img_tuple):
  target, input_image, mask = img_tuple

  predicted = generator([input_image,mask], training=True)
  predicted = gen_to_label(predicted)
  predicted = predicted * mask + target*(1-mask)
  target = tf.expand_dims(target, 0)
  
  #convert to rgb images
  target = label_to_rgb(target)
  input_image = label_to_rgb(input_image)
  predicted = label_to_rgb(predicted)

  images = [input_image[0], target[0], predicted[0]]
  title = ["Input Image", "Ground Truth", "Predicted Image"]
  plt.figure(figsize=(15, 15))

  for i, image in enumerate(images):
    plt.subplot(3,1,i+1)
    plt.title(title[i])
    plt.imshow(image*0.5+0.5)
    plt.axis('off')
  plt.show()

# SemGAN

## create new dataset

In [ ]:
def load_images(image_file, semantics_file):
  image = load(image_file)
  image = resize(image, IM_HEIGHT, IM_WIDTH)
  masked_image, mask = image_mask(image, 64, 32, 4)
  image = normalize(image)
  masked_image = normalize(masked_image)
  masked_image = tf.cast(masked_image, tf.float32)

  mask_int = tf.cast(mask, tf.int32)
  semantics = tf.io.read_file(semantics_file)
  semantics = tf.io.decode_png(semantics, channels = 0)
  semantics = tf.cast(semantics, tf.int32)
  semantics = resize(semantics, 256, 512)
  semantics_masked = (1-mask_int)*semantics
  semantics_masked = tf.cast(semantics_masked, tf.float32)


  return masked_image, image, semantics_masked, semantics, mask, image_file

In [ ]:
# Fill pandas dataframe with the paths to the files
def folder_to_pd(image_dir, semantics_dir):
  image_paths = {"test": [], "train": [], "val": []}
  semantics_paths = {"test": [], "train": [], "val": []}

  for data_type in ["test", "train", "val"]:
    image_subdir = os.path.join(image_dir, data_type)
    for root, subdir, files in os.walk(image_subdir):
      subdir.sort()
      files.sort()
      if files:
        for file in files:
          image_paths[data_type].append(os.path.join(root, file))

  for data_type in ["test", "train", "val"]:
    semantics_subdir = os.path.join(semantics_dir, data_type)
    for root, subdir, files in os.walk(semantics_subdir):
      subdir.sort()
      files.sort()
      if files:
        for i,file in enumerate(files):
          if file[-12:] == "labelIds.png":
            semantics_paths[data_type].append(os.path.join(root, file))


    
  test_ds = pd.DataFrame(list(zip(image_paths["test"], semantics_paths["test"])), columns =['Images', 'Semantics'])
  train_ds = pd.DataFrame(list(zip(image_paths["train"], semantics_paths["train"])), columns =['Images', 'Semantics'])
  val_ds = pd.DataFrame(list(zip(image_paths["val"], semantics_paths["val"])), columns =['Images', 'Semantics'])

  return test_ds, train_ds, val_ds

test_pd, train_pd, val_pd = folder_to_pd("/content/cityScapes/img/leftImg8bit", "/content/cityScapes/annotations/gtFine")

test_ds = tf.data.Dataset.from_tensor_slices((test_pd["Images"], test_pd["Semantics"]))
train_ds = tf.data.Dataset.from_tensor_slices((train_pd["Images"], train_pd["Semantics"]))
val_ds = tf.data.Dataset.from_tensor_slices((val_pd["Images"], val_pd["Semantics"]))

tf_train = train_ds.shuffle(train_ds.cardinality(), reshuffle_each_iteration=False)
#tf_train = tf_train.interleave(tf.data.TFRecordDataset, num_parallel_calls = tf.data.AUTOTUNE)
tf_train = tf_train.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
tf_train = tf_train.batch(BATCH_SIZE)
tf_train = tf_train.cache("/content/temporary2.tfcache")
tf_train = tf_train.prefetch(tf.data.AUTOTUNE)

tf_val = val_ds.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
tf_val = tf_val.batch(BATCH_SIZE)
tf_val = tf_val.prefetch(tf.data.AUTOTUNE)

## restore semantics GAN

In [ ]:
generator_semantics = GeneratorSemantics()
discriminator_semantics = DiscriminatorSemantics()

In [ ]:
checkpoint_dir_semantics = '/content/drive/MyDrive/Scriptie/ckpt/SemanticInpainting'
checkpoint_prefix_semantics = os.path.join(checkpoint_dir_semantics, "semantics", "ckpt")
checkpoint_semantics = tf.train.Checkpoint(generator_optimizer=generator_optimizer_semantics,
                                 discriminator_optimizer=discriminator_optimizer_semantics,
                                 generator=generator_semantics,
                                 discriminator=discriminator_semantics)

In [ ]:
checkpoint.restore('/content/drive/MyDrive/Scriptie/ckpt/SemanticInpainting/semantics/ckpt-1')

##GAN

In [ ]:
def Generator():
        down_stack = [
            downsample(64, 4, apply_batchnorm=False),  # (bs, 128, 256, 64)
            downsample(128, 4),  # (bs, 64, 128, 128)
            downsample(256, 4),  # (bs, 32, 64, 256)
            downsample(512, 4),  # (bs, 16, 32, 512)
            downsample(512, 4),  # (bs, 8, 16, 512)
            downsample(512, 4),  # (bs, 4, 8, 512)
            downsample(512, 4),  # (bs, 2, 4, 512)
            downsample(512, 4),  # (bs, 1, 2, 512)
        ]

        up_stack = [
            upsample(512, 4, 1, 2, apply_dropout=True),  # (bs, 2, 4, 1024)
            upsample(512, 4, 2, 4, apply_dropout=True),  # (bs, 4, 8, 1024)
            upsample(512, 4, 4, 8, apply_dropout=True),  # (bs, 8, 16, 1024)
            upsample(512, 4, 8, 16),  # (bs, 16, 32, 1024)
            upsample(256, 4, 16, 32),  # (bs, 32, 64, 512)
            upsample(128, 4, 32, 64),  # (bs, 64, 128, 256)
            upsample(64, 4, 64, 128),  # (bs, 128, 256, 128)
        ]

        initializer = tf.random_normal_initializer(0., 0.02)
        last_upsample = upsample(64, 4, 128, 256)
        last = tf.keras.layers.Conv2D(3, 4, strides=1, padding='same', kernel_initializer=initializer, activation='tanh')  # (bs, 256, 512, 35) 35 is the number of classes 

        concat = tf.keras.layers.Concatenate()
        image = tf.keras.layers.Input(shape=[IM_HEIGHT, IM_WIDTH, 3])
        semantics = tf.keras.layers.Input(shape=[IM_HEIGHT, IM_WIDTH, 35])

        x = tf.keras.layers.concatenate([image, semantics])

        # Downsampling through the model
        skips = []
        for down in down_stack:
            x = down(x)
            skips.append(x)

        skips = reversed(skips[:-1])

        # Upsampling and establishing the skip connections
        for up, skip in zip(up_stack, skips):
            x = up(x)
            x = concat([x, skip])

        x = last_upsample(x)
        x = last(x)

        return tf.keras.Model(inputs=[image, semantics], outputs=x)

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)

    image = tf.keras.layers.Input(shape=[IM_HEIGHT, IM_WIDTH, 3], name='input_image')
    semantics = tf.keras.layers.Input(shape=[IM_HEIGHT, IM_WIDTH, 35], name='semantics')
    tar = tf.keras.layers.Input(shape=[IM_HEIGHT, IM_WIDTH, 3], name='target_image')

    x = tf.keras.layers.concatenate([image, semantics, tar])  # (bs, 256, 512, channels*2)

    down1 = downsample(64, 4, False)(x)  # (bs, 128, 256, 64)
    down2 = downsample(128, 4)(down1)  # (bs, 64, 128, 128)
    down3 = downsample(256, 4)(down2)  # (bs, 32, 64, 256)

    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (bs, 34, 66, 256)
    conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                  kernel_initializer=initializer,
                                  use_bias=False)(zero_pad1)  # (bs, 31, 63, 512)

    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (bs, 33, 65, 512)

    last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                  kernel_initializer=initializer)(zero_pad2)  # (bs, 30, 62, 1)

    return tf.keras.Model(inputs=[image, semantics, tar], outputs=last)

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
    total_disc_loss = real_loss + generated_loss

    return total_disc_loss

def generator_loss( disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output)) # mean absolute error
    total_gen_loss = gan_loss + (LAMBDA * l1_loss)

    return total_gen_loss, gan_loss, l1_loss

In [ ]:
generatorSEMGAN = Generator()
discriminatorSEMGAN = Discriminator()

In [ ]:
generatorSEMGAN_GT = Generator()
discriminatorSEMGAN_GT = Discriminator()

In [ ]:
lr = 2e-4
generator_optimizer = tf.keras.optimizers.Adam(lr, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(lr, beta_1=0.5)

In [ ]:
checkpoint_dir_SEMGAN = '/content/drive/MyDrive/Scriptie/ckpt/SEMGAN/'
checkpoint_prefix_SEMGAN = os.path.join(checkpoint_dir_SEMGAN, "ckpt", "sparse")
checkpoint_SEMGAN = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generatorSEMGAN,
                                 discriminator=discriminatorSEMGAN)

In [ ]:
checkpoint.restore('/content/drive/MyDrive/Scriptie/ckpt/SEMGAN/ckpt-2')

## Train

In [ ]:
def generate_images(model, masked_input, tar, semantics_prediction, semantics_sparse, mask):
  prediction = model([masked_input, semantics_sparse], training=True)
  prediction = prediction*mask+tar*(1-mask)
  plt.figure(figsize=(15, 15))
  semantics_prediction = label_to_rgb(semantics_prediction)

  display_list = [masked_input[0], tar[0], prediction[0], semantics_prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image', 'Input Semantics']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

### FIT SEMGAN

In [ ]:
@tf.function
def train_epochSEMGAN(input_image, target, semantics, mask):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generatorSEMGAN([input_image, semantics], training=True)
    gen_output = gen_output*mask+ target*(1-mask) #combine generated patches with valid pixels from ground truth

    disc_real_output = discriminatorSEMGAN([input_image, semantics, target], training=True)
    disc_generated_output = discriminatorSEMGAN([input_image, semantics, gen_output], training=True) #discriminator takes predicted labels

    gen_total_loss, gen_gan_loss, gen_L1_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generatorSEMGAN.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                              discriminatorSEMGAN.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients, generatorSEMGAN.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminatorSEMGAN.trainable_variables))
  
  return gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss
    
  
def getValAccLossSEMGAN(input_image, target, semantics, mask): 
  gen_output = generatorSEMGAN([input_image, semantics], training=True)
  gen_output = gen_output*mask+ target*(1-mask) #convert the 35 channel output of the generator to a 1 channel output with labels

  disc_real_output = discriminatorSEMGAN([input_image, semantics, target], training=True)
  disc_generated_output = discriminatorSEMGAN([input_image, semantics, gen_output], training=True) #discriminator takes predicted labels

  gen_total_loss, gen_gan_loss, gen_L1_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
  disc_loss = discriminator_loss(disc_real_output, disc_generated_output)


  return gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss

In [ ]:
log_dir= '/content/drive/MyDrive/Scriptie/logs/SEMGAN/epoch/'
summary_writer_train = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_Train")
summary_writer_val = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_Val")

In [ ]:
def fitSEMGAN(train_ds, test_ds, epochs):
  masked_image_test, image_test, semantics_masked_test, semantics_test, mask_test, image_file_test = next(iter(test_ds.take(1)))
  start = time.time()
  total_ims = tf.cast(train_ds.cardinality(), tf.float32)
  total_ims_val = tf.cast(test_ds.cardinality(), tf.float32)

  for epoch in range(epochs):
    metrics_dict = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    
    #semantics used for the example
    semantics_predicted_test = gen_to_label(generator_semantics(semantics_masked_test, training=True)) # turn the generator output into labels
    semantics_predicted_test = semantics_predicted_test*tf.cast(mask_test, tf.int32)+semantics_test*(1-tf.cast(mask_test, tf.int32)) # combine the prediction with the ground truth
    semantics_sparse_test = label_to_sparse(semantics_predicted_test) # make a sparse matrix as input to the generator
    generate_images(generatorSEMGAN, masked_image_test, image_test, semantics_predicted_test, semantics_sparse_test, mask_test)

    print("Start training of epoch: ", epoch+1)
    for step,(masked_image, image, semantics_masked, semantics, mask, image_file) in train_ds.enumerate():
      semantics_predicted = gen_to_label(generator_semantics(semantics_masked, training=True)) # turn the generator output into labels
      semantics_predicted = semantics_predicted * tf.cast(mask, tf.int32) + semantics *(1-tf.cast(mask, tf.int32)) # combine the prediction with the ground truth
      semantics_predicted = label_to_sparse(semantics_predicted) # make a sparse matrix as input to the generator
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = train_epochSEMGAN(masked_image, image, semantics_predicted, mask)
      metrics_list = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
      
      for i,(key,value) in enumerate(metrics_dict.items()):
        metrics_dict[key] = value + metrics_list[i]
      
      if (step+1) % 50 == 0:
        print('.', end='', flush=True)

    with summary_writer_train.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict["gen_total_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict["gen_gan_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict["gen_L1_loss"]/total_ims, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict["disc_loss"]/total_ims, step=epoch)


    # Gather the metrics for the validation set
    metrics_dict_val = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    print("Gathering validation set metrics...")
    for step,(masked_image, image, semantics_masked, semantics, mask, image_file) in test_ds.enumerate():
      semantics_predicted = gen_to_label(generator_semantics(semantics_masked, training=True))
      semantics_predicted = semantics_predicted * tf.cast(mask, tf.int32) + semantics *(1-tf.cast(mask, tf.int32))
      semantics_predicted = label_to_sparse(semantics_predicted)
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = getValAccLossSEMGAN(masked_image, image, semantics_predicted, mask)
      metrics_list_val = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
        
      for i,(key,value) in enumerate(metrics_dict_val.items()):
        metrics_dict_val[key] = value + metrics_list_val[i]
    
    with summary_writer_val.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict_val["gen_total_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict_val["gen_gan_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict_val["gen_L1_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict_val["disc_loss"]/total_ims_val, step=epoch)

    display.clear_output(wait=True)
    print(f'Time taken for 1 epoch: {time.time()-start:.2f} sec\n')
    start = time.time()

    # Save (checkpoint) the model every 5 epochs
    #if epoch % 10 == 0 & epoch !=0: 
    #  checkpoint.save(file_prefix=checkpoint_prefix)

In [ ]:
fitSEMGAN(tf_train, tf_val, 10)

### FIT SEMGAN-GT

In [ ]:
@tf.function
def train_epochSEMGAN_GT(input_image, target, semantics, mask):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generatorSEMGAN_GT([input_image, semantics], training=True)
    gen_output = gen_output*mask+ target*(1-mask) #combine generated patches with valid pixels from ground truth

    disc_real_output = discriminatorSEMGAN_GT([input_image, semantics, target], training=True)
    disc_generated_output = discriminatorSEMGAN_GT([input_image, semantics, gen_output], training=True) #discriminator takes predicted labels

    gen_total_loss, gen_gan_loss, gen_L1_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generatorSEMGAN_GT.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                              discriminatorSEMGAN_GT.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients, generatorSEMGAN_GT.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminatorSEMGAN_GT.trainable_variables))
  
  return gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss
    
  
def getValAccLossSEMGAN_GT(input_image, target, semantics, mask): 
  gen_output = generatorSEMGAN_GT([input_image, semantics], training=True)
  gen_output = gen_output*mask+ target*(1-mask) #convert the 35 channel output of the generator to a 1 channel output with labels

  disc_real_output = discriminatorSEMGAN_GT([input_image, semantics, target], training=True)
  disc_generated_output = discriminatorSEMGAN_GT([input_image, semantics, gen_output], training=True) #discriminator takes predicted labels

  gen_total_loss, gen_gan_loss, gen_L1_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
  disc_loss = discriminator_loss(disc_real_output, disc_generated_output)


  return gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss

In [ ]:
def fitSEMGAN_GT(train_ds, test_ds, epochs):
  masked_image_test, image_test, semantics_masked_test, semantics_test, mask_test, image_file_test = next(iter(test_ds.take(1)))
  start = time.time()
  total_ims = tf.cast(train_ds.cardinality(), tf.float32)
  total_ims_val = tf.cast(test_ds.cardinality(), tf.float32)

  for epoch in range(epochs):
    metrics_dict = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    
    #semantics used for the example
    semantics_predicted_test = gen_to_label(generator_semantics(semantics_masked_test, training=True)) # turn the generator output into labels
    semantics_predicted_test = semantics_predicted_test*tf.cast(mask_test, tf.int32)+semantics_test*(1-tf.cast(mask_test, tf.int32)) # combine the prediction with the ground truth
    semantics_sparse_test = label_to_sparse(semantics_predicted_test) # make a sparse matrix as input to the generator
    generate_images(generatorSEMGAN_GT, masked_image_test, image_test, semantics_predicted_test, semantics_sparse_test, mask_test)

    print("Start training of epoch: ", epoch+1)
    for step,(masked_image, image, semantics_masked, semantics, mask, image_file) in train_ds.enumerate():
      semantics_predicted = gen_to_label(generator_semantics(semantics_masked, training=True)) # turn the generator output into labels
      semantics_predicted = semantics_predicted * tf.cast(mask, tf.int32) + semantics *(1-tf.cast(mask, tf.int32)) # combine the prediction with the ground truth
      semantics_predicted = label_to_sparse(semantics_predicted) # make a sparse matrix as input to the generator
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = train_epochSEMGAN_GT(masked_image, image, semantics_predicted, mask)
      metrics_list = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
      
      for i,(key,value) in enumerate(metrics_dict.items()):
        metrics_dict[key] = value + metrics_list[i]
      
      if (step+1) % 50 == 0:
        print('.', end='', flush=True)

    with summary_writer_train.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict["gen_total_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict["gen_gan_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict["gen_L1_loss"]/total_ims, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict["disc_loss"]/total_ims, step=epoch)


    # Gather the metrics for the validation set
    metrics_dict_val = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    print("Gathering validation set metrics...")
    for step,(masked_image, image, semantics_masked, semantics, mask, image_file) in test_ds.enumerate():
      semantics_predicted = gen_to_label(generator_semantics(semantics_masked, training=True))
      semantics_predicted = semantics_predicted * tf.cast(mask, tf.int32) + semantics *(1-tf.cast(mask, tf.int32))
      semantics_predicted = label_to_sparse(semantics_predicted)
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = getValAccLossSEMGAN_GT(masked_image, image, semantics_predicted, mask)
      metrics_list_val = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
        
      for i,(key,value) in enumerate(metrics_dict_val.items()):
        metrics_dict_val[key] = value + metrics_list_val[i]
    
    with summary_writer_val.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict_val["gen_total_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict_val["gen_gan_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict_val["gen_L1_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict_val["disc_loss"]/total_ims_val, step=epoch)

    display.clear_output(wait=True)
    print(f'Time taken for 1 epoch: {time.time()-start:.2f} sec\n')
    start = time.time()

    # Save (checkpoint) the model every 5 epochs
    #if epoch % 10 == 0 & epoch !=0: 
    #  checkpoint.save(file_prefix=checkpoint_prefix)

In [ ]:
fitSEMGAN_GT(tf_train, tf_val, 10)

In [ ]:
print(checkpoint_prefix_SEMGAN)

In [ ]:
checkpoint.save(file_prefix=checkpoint_prefix_SEMGAN)